In [36]:
import pandas as pd
import mne as mne
import os 
import time
import numpy as np
import matplotlib.pyplot as plt
import joblib
import constants
from IPython.utils import io
import time
import sys
import yasa
from scipy.signal import welch

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score

#Import my modules
import format_eeg_data
import constants
import eeg_stat_ts
import run_expts


from mne_connectivity import spectral_connectivity_epochs
from mne.datasets import sample
import seaborn as sns

from scipy.signal import welch
import yasa
import constants
import numpy as np

#PCA Imports
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier

#Pipeline
from sklearn.pipeline import Pipeline

In [2]:
power_bands = {'delta' : (0.5,4) , 'theta' : (4,8) , 'alpha' : (8,12) , 'sigma' : (12,16) , 'beta' : (16,30) , 'gamma' : (30,40) }
power_band_names = [val for val in power_bands.keys()]
data_type = 'REM'
folder = '/user/home/ko20929/work/RBD_using_custom_package/Blue_pebble/Connectivity/'
X = pd.read_hdf(folder + data_type+ '_coherence_df.h5')
X = X.reset_index(drop = True)
y = pd.read_hdf('/user/home/ko20929/work/RBD_using_custom_package/Blue_pebble/C_22_data/Full_dfs/' + data_type + '_y.h5')
groups = pd.read_hdf('/user/home/ko20929/work/RBD_using_custom_package/Blue_pebble/C_22_data/Full_dfs/' + data_type + '_groups.h5')

In [7]:
#Lets do a PCA with 10 components
pca = PCA(n_components = 10)
scaler = StandardScaler()
clf = DecisionTreeClassifier()
pipe = Pipeline(steps=[("scaler", scaler), ("pca", pca), ("decision_tree", clf)])
clf_dict = {'DT_PCA_10': pipe}

In [39]:
#Lets do a PCA with 10 components ________________________________________________________________________________________________
t1 = time.time()

pca = PCA(n_components = 12)
scaler = StandardScaler()
clf = DecisionTreeClassifier()
pipe = Pipeline(steps=[("scaler", scaler), ("pca", pca), ("decision_tree", clf)])
pipe_2 = Pipeline(steps=[("scaler", scaler), ("pca", pca), ("R_F", RandomForestClassifier())])
clf_dict = {'DT_PCA': pipe, 'RF_PCA': pipe_2}

## Try with LDA and QDA
clf = QuadraticDiscriminantAnalysis()
clf = LinearDiscriminantAnalysis()

pipe = Pipeline(steps=[("scaler", scaler), ("lda", LinearDiscriminantAnalysis())])
pipe_2 = Pipeline(steps=[("scaler", scaler), ("qda", QuadraticDiscriminantAnalysis())])
clf_dict = {'lda': pipe, 'qda': pipe_2}


#Define subsets of features to be used in Experiments ___________________________________________________________________________
subsets_dict = {}
for p_band in power_band_names:
    subsets_dict[p_band] = [col for col in X.columns if p_band in col]
subsets_dict['all_features'] = [col for col in X.columns]

X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(X,y,groups, 4)

results_df = run_expts.run_mv_tsc(X_expt,y_expt,groups_expt, clf_dict, return_df = True , subset_names_and_cols = subsets_dict, random_states = [1,2] )

t2 = time.time()

print(t2-t1)

acc_std_df = run_expts.generate_subset_acc_std(results_df , return_df = True)

acc_std_df[[col for col in acc_std_df.columns if 'acc' in col]].mean(axis = 0)

/user/home/ko20929/.conda/envs/sktime_latest/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/user/home/ko20929/.conda/envs/sktime_latest/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/user/home/ko20929/.conda/envs/sktime_latest/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/user/home/ko20929/.conda/envs/sktime_latest/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/user/home/ko20929/.conda/envs/sktime_latest/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/user/home/ko20929/.conda/envs/sktime_la

19.60263419151306


delta_lda_acc           0.346703
delta_qda_acc           0.295604
theta_lda_acc           0.413736
theta_qda_acc           0.295604
alpha_lda_acc           0.391758
alpha_qda_acc           0.253846
sigma_lda_acc           0.355495
sigma_qda_acc           0.218681
beta_lda_acc            0.275824
beta_qda_acc            0.210440
gamma_lda_acc           0.413736
gamma_qda_acc           0.245055
all_features_lda_acc    0.384615
all_features_qda_acc    0.201099
dtype: float64

In [32]:
PCA_20_results_expt_2

delta_DT_PCA_acc           0.463333
delta_RF_PCA_acc           0.516667
theta_DT_PCA_acc           0.567778
theta_RF_PCA_acc           0.492222
alpha_DT_PCA_acc           0.468889
alpha_RF_PCA_acc           0.554444
sigma_DT_PCA_acc           0.534444
sigma_RF_PCA_acc           0.448889
beta_DT_PCA_acc            0.586667
beta_RF_PCA_acc            0.617778
gamma_DT_PCA_acc           0.640000
gamma_RF_PCA_acc           0.564444
all_features_DT_PCA_acc    0.523333
all_features_RF_PCA_acc    0.594444
dtype: float64

In [28]:
PCA_20_results_expt_1

delta_DT_PCA_acc           0.5375
delta_RF_PCA_acc           0.5875
theta_DT_PCA_acc           0.6250
theta_RF_PCA_acc           0.6000
alpha_DT_PCA_acc           0.5625
alpha_RF_PCA_acc           0.6250
sigma_DT_PCA_acc           0.5250
sigma_RF_PCA_acc           0.5625
beta_DT_PCA_acc            0.6125
beta_RF_PCA_acc            0.5875
gamma_DT_PCA_acc           0.6125
gamma_RF_PCA_acc           0.6750
all_features_DT_PCA_acc    0.5500
all_features_RF_PCA_acc    0.6500
dtype: float64